<a href="https://colab.research.google.com/github/kai-nat/CS-370-Artificial-Intelligence/blob/main/ADS2_7PAM2001_0901_Assignment_1_Deep_Learning_with_Keras_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Module imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D

In [2]:
### Create a class called latent_sampling, which subclasses layers.Layer.
### The class should perform the reparameterisation trick in its .call()
### method.
### Reparameterization Trick: z = mean + epsilon * exp(ln(variance) * 0.5)
### epsilon = N(0,1), a unit normal with same dims as mean and variance

# Include the follow two lines in your .call method:
# self.add_loss(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)))
# self.add_metric(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)), name='kl_loss'

class latent_sampling(layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(latent_sampling, self).__init__()
        self.units = units
        self.input_dim = input_dim
        self.mean = Dense(self.units, activation='relu')
        self.logvar = Dense(self.units, activation='relu')
        self.epsilon = tf.random.normal(shape=(self.input_dim, self.units))
        self.z = self.mean + tf.exp(0.5 * self.logvar) * self.epsilon
        self.add_loss(-0.5 * tf.reduce_sum(1 + self.logvar - tf.square(self.mean) - tf.exp(self.logvar)))
        self.add_metric(-0.5 * tf.reduce_sum(1 + self.logvar - tf.square(self.mean) - tf.exp(self.logvar)), name='kl_loss')
    
    def call(self, inputs):
        return self.z

In [3]:
### Create the encoder model, using the functional API and the architecture
### detailed below. Use tf.keras.models.Model to initialise the model.

# Model: "encoder"
# ____________________________________________________________________________________________________
#  Layer (type)            Output Shape           Activation  kernel_size  padding  Input
# ====================================================================================================
#  enc_input (InputLayer)  [(None, 128, 128, 3)]  None
#  enc_conv_1 (Conv2D)     (None, 64, 64, 32)     ReLU        (3,3)        'same'   enc_input
#  enc_conv_2 (Conv2D)     (None, 32, 32, 64)     ReLU        (3,3)        'same'   enc_conv_1
#  enc_conv_3 (Conv2D)     (None, 16, 16, 64)     ReLU        (3,3)        'same'   enc_conv_2    
#  enc_conv_4 (Conv2D)     (None, 8, 8, 64)       ReLU        (3,3)        'same'   enc_conv_3    
#  enc_flat (Flatten)      (None, 4096)           None        None         None     enc_conv_4
#  z_mean (Dense)          (None, 200)            None        None         None     enc_flat                        
#  z_log_var (Dense)       (None, 200)            None        None         None     enc_flat
#  z (latent_sampling)     (None, 200)            None        None         None     (z_mean, z_log_var)

def encoder_model():
    enc_input = layers.Input(shape=(128, 128, 3), name='enc_input')
    enc_conv_1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='enc_conv_1')(enc_input)
    enc_conv_2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='enc_conv_2')(enc_conv_1)
    enc_conv_3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='enc_conv_3')(enc_conv_2)
    enc_conv_4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='enc_conv_4')(enc_conv_3)
    enc_flat = layers.Flatten(name='enc_flat')(enc_conv_4)
    z_mean = layers.Dense(200, name='z_mean')(enc_flat)
    z_log_var = layers.Dense(200, name='z_log_var')(enc_flat)
    z = latent_sampling(200, 200)([z_mean, z_log_var])
    return keras.Model(enc_input, z, name='encoder')

In [4]:
### Create the decoder model, using the functional API and the architecture
### detailed below. Use tf.keras.models.Model to initialise the model.

# Model: "decoder"
# ____________________________________________________________________________________________________
#  Layer (type)                   Output Shape           Activation  kernel_size  padding  Input
# ====================================================================================================
#  dec_input (InputLayer)         [(None, 200)]          None
#  dec_dense (Dense)              (None, 4096)           ReLU        None         None     dec_input
#  dec_reshape (Reshape)          (None, 8, 8, 64)       None        None         None     dec_dense
#  dec_conv_1 (Conv2DTranspose)   (None, 8, 8, 64)       ReLU        (3,3)        'same'   dec_reshape
#  dec_conv_2 (Conv2DTranspose)   (None, 16, 16, 64)     ReLU        (3,3)        'same'   dec_conv_1
#  dec_conv_3 (Conv2DTranspose)   (None, 32, 32, 64)     ReLU        (3,3)        'same'   dec_conv_2    
#  dec_conv_4 (Conv2DTranspose)   (None, 64, 64, 32)     ReLU        (3,3)        'same'   dec_conv_3    
#  dec_output (Conv2DTranspose)   (None, 128, 128, 3)    ReLU        (3,3)        'same'   dec_conv_4

def decoder_model():
    dec_input = layers.Input(shape=(200,), name='dec_input')
    dec_dense = layers.Dense(4096, activation='relu', name='dec_dense')(dec_input)
    dec_reshape = layers.Reshape((8, 8, 64), name='dec_reshape')(dec_dense)
    dec_conv_1 = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same', name='dec_conv_1')(dec_reshape)
    dec_conv_2 = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same', name='dec_conv_2')(dec_conv_1)
    dec_conv_3 = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same', name='dec_conv_3')(dec_conv_2)
    dec_conv_4 = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same', name='dec_conv_4')(dec_conv_3)
    dec_output = layers.Conv2DTranspose(3, (3, 3), activation='relu', padding='same', name='dec_output')(dec_conv_4)
    return keras.Model(dec_input, dec_output, name='decoder')


In [1]:
### Create the VAE model, again using tf.keras.models.Model, with the function
### API to combine the feed the outputs of the encoder into the inputs of the
### decoder.

# Model: "vae"

def vae_model():
    encoder = encoder_model()
    decoder = decoder_model()
    vae_input = layers.Input(shape=(128, 128, 3), name='vae_input')
    z = encoder(vae_input)
    vae_output = decoder(z)
    return keras.Model(vae_input, vae_output, name='vae')


In [6]:
# Provided here are the loss functions for the VAE model.

def recon_loss(y_true, y_pred):
    recon = tf.reduce_sum(tf.square(y_true-y_pred), axis=(1,2,3))
    return tf.reduce_mean(recon)

In [ ]:
# This code is provided to load a subsample of the celeb_a dataset, and process
# the images into the correct format for the model.

import tensorflow_datasets as tfds

def img_process(features):
    """
    A preprocessing fuction for the test and validation datasets. This function
    accepts the oxford_iiit_pet dataset, extracts the images and species label,
    and resizes and rescales the images.
    """
    image = tf.image.resize(features['image'], (128,128))
    image = tf.cast(image, 'float32')/255.
    return image, image


train_ds, test_ds = tfds.load('celeb_a', split=['train[:10%]', 'test[:10%]'], download=True, shuffle_files=True)
train_ds = train_ds.map(img_process).cache().batch(64)
test_ds = test_ds.map(img_process).cache().batch(64)


In [4]:
### Compile the VAE model, choosing an appropriate optimizer and learning rate,
### the total_loss function as the model loss, and any appropriate metrics.

vae = vae_model()
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=recon_loss, metrics=['mse'])



NameError: ignored

In [5]:
### Train the model using the train dataset for an appropriate number of epochs

vae.fit(train_ds, epochs=10, validation_data=test_ds)

### Store the losses and metrics in the history dictionary

history = vae.history.history



NameError: ignored

In [7]:
### Plot the losses and metrics. Comment on the figures in your report, with
### regard to how the training has proceeded.

import matplotlib.pyplot as plt

plt.plot(history['loss'], label='loss')
plt.plot(history['val_loss'], label='val_loss')
plt.legend()
plt.show()

plt.plot(history['mse'], label='mse')
plt.plot(history['val_mse'], label='val_mse')
plt.legend()
plt.show()


NameError: ignored

In [8]:
### Using the test dataset, create a plot that shows the reconstruction quality
### of the training model. Comment on the results in your report.

def plot_recon(model, test_ds):
    test_images = next(iter(test_ds))[0]
    recon_images = model(test_images)
    fig, axes = plt.subplots(2, 5, figsize=(10, 5))
    for i in range(5):
        axes[0, i].imshow(test_images[i])
        axes[1, i].imshow(recon_images[i])
    plt.show()
    
plot_recon(vae, test_ds)



NameError: ignored

In [9]:
### Demonstrate the generative properties of the VAE by drawing randomly sampled
### latent vectors from a unit Guassian and passing them to the train decoder.
### Plot the results and comment on them in your report.

import numpy as np

z = np.random.normal(size=(5, 200))
recon_images = decoder_model()(z)

fig, axes = plt.subplots(1, 5, figsize=(10, 4))

for i in range(5):
    axes[i].imshow(recon_images[i])

plt.show()

NameError: ignored